# 基盤人工知能演習 第7回

※本演習資料の二次配布・再配布はお断り致します。

　今回の演習資料は以下の2つのテーマが記載されている。

　**AI7.1 | 畳み込みニューラルネットワーク (CNN)**

　**AI7.2 | 再帰型ニューラルネットワーク (RNN)**

　なお、時間の都合上、本日の授業時間内では**AI7.1 | 畳み込みニューラルネットワーク (CNN)** のみを取り扱い、再帰型ニューラルネットワークは資料のみ配布する。


## AI7.1 | 畳み込みニューラルネットワーク (Convolutional Neural Network; CNN) 

　CNNを使わない画像ベースの学習手法は無いのではないか？というレベルに広く用いられている手法である。前回の演習でも利用したMNISTの分類予測を行うことで、CNNがどのくらい強力か見てみよう。

### AI7.1.1 | CNNの構成要素のおさらい

　2次元のCNNは`torch.nn.Conv2d(16, 32, kernel_size=(5, 5), padding=2, stride=2)`のように定義するのだが、見てわかるようにこれまでの`torch.nn.Linear(512, 512)`に比べると設定すべきパラメータが多い。
これから自分が組むニューラルネットワークがどのような計算を行っているかを理解することは大切なので、講義でも習った要素を今一度確認しよう。

#### 畳み込み演算
　画像処理における「畳み込み」という操作は、**図AI7.1**のように、入力行列 $f$ とフィルタ $g$ を入力として、**フィルタの適用範囲を平行移動させながら $f$ と $g$ の積和を計算していく処理**であり、一般的には **$f*g$** と記述する。なお、後述のパディングを行わない限り出力の画像サイズは入力の画像サイズよりも小さくなることに注意しよう。

　この演算において、学習する重み $w$ はどこにあるだろうか。**フィルタ $g$ の各要素が学習によって最適化される重み**である。

![図AI7.1](https://i.imgur.com/GTQj9dH.png)

**図AI7.1 | 畳み込み演算**


#### 入力チャネル数と出力チャネル数

　最も単純なCNNは各ピクセルごとに1つの値が存在するデータを処理する例である。これは、グレースケールの画像（例えばこれまでやってきたMNIST）を入力とする時などが考えられる。
一方、例えばカラーの画像はRGB (Red, Green, Blue) の3つのデータが各ピクセルに与えられており、これは**3チャネルの入力**ということができる。この場合は、**図AI7.2**のように1つの出力値を作るために3倍の計算を行う必要がある。

　さらに、フィルタを複数用意することで出力チャネル数も複数にすることができる。例えば、**図AI7.3**は**入力チャネル数が3、出力チャネル数が2**であるようなCNNの例になっている。

　`torch.nn.Conv2d(16, 32, kernel_size=(5, 5), padding=2, stride=2)`のうち、16と32の数値がそれぞれ入力/出力チャネル数である。



![図AI7.2](https://i.imgur.com/y3t7aw6.png)

**図AI7.2 | 複数の入力チャネルが存在するCNN** 基盤人工知能 第7回 講義資料より抜粋・一部改変

![図AI7.3](https://i.imgur.com/mBXnaEY.png)

**図AI7.3 | 入力チャネルと出力チャネルが複数であるCNN** 基盤人工知能 第7回 講義資料より抜粋・一部改変


#### フィルタサイズ（カーネルサイズ）
　`torch.nn.Conv2d(16, 32, kernel_size=(5, 5), padding=2, stride=2)` の**`kernel_size=(5,5)`**で与えている情報、これがフィルタサイズ（あるいはカーネルサイズ）である。フィルタサイズは1つの値を出力するための**積和計算を行う範囲を示す値**で、例えば`(2,2)`であれば合計4ピクセルについて、入力データとフィルタとの積和を計算することになる（前述の**図AI7.1**の例はフィルタサイズが2×2となっている）。

#### パディング

　`torch.nn.Conv2d(16, 32, kernel_size=(5, 5), padding=2, stride=2)` の **`padding=2`** がパディングである。
パディングが無いと前述のように出力の画像サイズが小さくなってしまう。それを防ぐために、周囲を0で埋めることがしばしば行われる。これがパディングである（**図AI7.4**）。フィルタサイズの1辺が $2k+1$ であるとき、パディングを $k$ とすることで、入力の画像サイズと出力の画像サイズが等しくなる。

![図AI7.4](https://i.imgur.com/s2IWPpB.png)

**図AI7.4 | パディングが存在するCNN** 基盤人工知能 第7回 講義資料より抜粋・一部改変

#### ストライド

　かなり大きな画像に対して学習を行う場合、サイズの圧縮を狙って数ピクセルに1回だけ畳み込みを行うことがある。これがストライドであり、**`stride=2`**がそれにあたる（**図AI7.5**）。

　ただ、通常はストライドを増やすより、後述のMaxPoolingを利用するのが一般的である。

![図AI7.5](https://i.imgur.com/FYKrN2A.png)

**図AI7.5 | ストライドが2であるCNN** 基盤人工知能 第7回 講義資料より抜粋・一部改変



### AI7.1.2 | Max pooling（プーリング層）

　次に、CNNと併せて用いられることが多いmax pooling（プーリング層）について簡単に説明する。Pooling層は複数のピクセルを1つの値にまとめる操作を行うもので、最大値を取るmax poolingが通常用いられる。これにより、わずかな画像の平行移動があっても予測に影響しにくくなると言われている。
**図AI7.6**を見てわかるように、画像の1辺が1/2, 1/3の単位で小さくなるので、非常に大きな画像を学習する場合にはより大きなストライド幅のプーリング層を用いたり、多数のプーリング層を導入したりする。

　PyTorchでは、`torch.nn.MaxPool2d(2)`とすることで、ストライド2の二次元max poolingを行うことができる。

![図AI7.6](https://i.imgur.com/kUH7lra.png)

**図AI7.6 | Max Poolingの例** 基盤人工知能 第7回 講義資料より抜粋・一部改変

### AI7.1.3 | Flatten

　最後に、2次元状に並んだデータを（これまで学習に利用していたような）1次元の配列に直す `torch.nn.Flatten()` について述べておく。 **`Flatten()` とは、図7.7のように、2次元（以上）のデータを1次元に変換する操作のこと**である。CNNを用いる学習では、最後の数層は`Linear()`層を用いることが多いため、2次元空間のピクセル様のデータから、これまで取り扱ってきた1次元のベクトルに変換する操作が必要になる。**CNN系の層とこれまで用いてきた層をつなぐ役割を果たしている**と考えればよいだろう。

![図 AI7.7](https://i.imgur.com/FAwNc0x.png)

**図 AI7.7 | Flattenの例** 複数チャネル存在している場合でも、1次元のベクトルに変換する。

-----
##### 課題 AI7.1

　入力画像が 2チャネル×縦10ピクセル×横12ピクセル である時、
`torch.nn.MaxPool2d(2)` によって得られる出力次元を答えよ。

-----

### AI7.1.4 | CNNを用いたMNISTの分類予測

　それでは、ここまでで説明してきたCNN, max pooling, flattenを利用して、手書き数字MNISTを学習してみよう。

　まずは、前回と同様にlivelosplotを用いるためのインストールを行う。Google Colabはクラウドサービスのため、毎回不足しているパッケージをインストールしたり、あるいはデータを作る必要があるので注意しよう。

In [0]:
# 以下のコマンドはPythonのコマンドではなく、Google Colabだから実行可能
# pipというPythonのパッケージ管理ツールを使ってlivelossplotをインストールしている
!pip install livelossplot

　次に、MNISTデータの準備を行う。
$X$の作成以外は前回と同じコードになっている。
$X$は、**1チャネル（モノクロ画像）×縦28ピクセル×横28ピクセル**のデータとして準備を行うため、
 `X_train` などは (データ数,1,28,28) の4階テンソルに整形する。


In [0]:
import numpy as np
import torch
import torchvision

trainset = torchvision.datasets.MNIST(root=".", train=True, download=True, transform=torchvision.transforms.ToTensor())
testset  = torchvision.datasets.MNIST(root=".", train=False, download=True, transform=torchvision.transforms.ToTensor())

In [0]:
# torch.tensor -> numpy.array
X_train = trainset.data.numpy()
y_train = trainset.targets.numpy()
X_test = testset.data.numpy()
y_test = testset.targets.numpy()

# scaled from [0,255] to [0,1] for X
X_train = X_train / 255
X_test  = X_test / 255

In [0]:
from sklearn.model_selection import train_test_split
# trainを50000件のtrainingデータと10000件のvalidationデータに分割する
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, 
                                                      train_size=50000, random_state=0)

# testはそのまま利用する

In [0]:
# ここが変更点
# 1チャネル×縦28ピクセル×横28ピクセル
X_train_fig = X_train.reshape(50000, 1, 28, 28) 
X_valid_fig = X_valid.reshape(10000, 1, 28, 28)
X_test_fig  = X_test.reshape(10000, 1, 28, 28) 

　あとは、いつもと同様に `torch.tensor` に変換させて、 DataLoader まで作ってしまおう。

In [0]:
batch_size = 256 # 今回は256件のデータごとに学習を行う

In [0]:
# training set
X_train_torch = torch.tensor(X_train_fig, dtype=torch.float) # dtype=torch.floatを忘れずに
y_train_torch = torch.tensor(y_train, dtype=torch.long)      # dtype=torch.longを忘れずに
train_dataset = torch.utils.data.TensorDataset(X_train_torch, y_train_torch)
train_loader  = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

In [0]:
# validation set
X_valid_torch = torch.tensor(X_valid_fig, dtype=torch.float)
y_valid_torch = torch.tensor(y_valid, dtype=torch.long)
valid_dataset = torch.utils.data.TensorDataset(X_valid_torch, y_valid_torch)
valid_loader  = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size)

In [0]:
# test set
X_test_torch = torch.tensor(X_test_fig, dtype=torch.float)
y_test_torch = torch.tensor(y_test, dtype=torch.long)
test_dataset = torch.utils.data.TensorDataset(X_test_torch, y_test_torch)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

　続いて、前回と同様にGPUの利用設定や、学習および予測のための関数群を定義する。前回の演習時のコードと全く変わらない。DataLoaderを使えるモデルであれば汎用的に使えるように関数は整備したので、実際の用途にも応用できるはずである。

In [0]:
import torch
from livelossplot import PlotLosses

device_GPU = torch.device("cuda:0") 

In [0]:
def update_model(model, loss_fn, opt, train_loader, device):
  train_loss = 0
  train_correct = 0
  train_count = len(train_loader.dataset)
  
  for X, y in train_loader:
    X = X.to(device) # GPUへデータを転送
    y = y.to(device) # GPUへデータを転送
    y_pred = model(X) # Xからyを予測
    
    _, predicted = torch.max(y_pred.data, 1) # 予測された10クラスの確率のうち、確率が最大だったものを得る
    train_correct += (predicted == y).sum().item() # 予測に成功した件数をカウント（accuracy計算用）
    
    loss = loss_fn(y_pred, y)        # ミニバッチ内の訓練誤差の 平均 を計算
    train_loss += loss.item()*len(y) # エポック全体の訓練誤差の 合計 を計算しておく
    
    # 重みの更新
    opt.zero_grad()
    loss.backward()
    opt.step()
    
  # エポック内の訓練誤差の平均値と予測精度を計算
  mean_train_loss = train_loss / train_count
  train_accuracy = train_correct / train_count
  
  return mean_train_loss, train_accuracy    

In [0]:
def evaluate_model(model, loss_fn, dataloader, device):
  model.eval() # 学習を行わない時は evaluate 状態にする （補足資料※1）

  valid_loss = 0
  valid_correct = 0
  valid_count = len(dataloader.dataset)

  for X, y in dataloader:
    X = X.to(device) # GPUへ転送
    y = y.to(device) # GPUへ転送
    y_pred = model(X) # Xからyを予測
    
    _, predicted = torch.max(y_pred.data, 1) # 予測された10クラスの確率のうち、確率が最大だったものを得る
    valid_correct += (predicted == y).sum().item() # 予測に成功した件数をカウント（accuracy計算用）
    
    loss = loss_fn(y_pred, y)        # ミニバッチ内の訓練誤差の 平均 を計算
    valid_loss += loss.item()*len(y) # エポック全体の訓練誤差の 合計 を計算しておく
    
  mean_valid_loss = valid_loss / valid_count
  valid_accuracy = valid_correct / valid_count

  model.train() # evaluate状態からtrain状態に戻しておく
  return mean_valid_loss, valid_accuracy

In [0]:
def train(model, loss_fn, opt, train_loader, valid_loader, device, epoch=50):
  liveloss = PlotLosses()
  for i in range(epoch):
    train_loss, train_accuracy = update_model(model, loss_fn, opt, train_loader, device)
    valid_loss, valid_accuracy = evaluate_model(model, loss_fn, valid_loader, device)
  
    # Visualize the loss and accuracy values.
    liveloss.update({
        'log loss': train_loss,
        'val_log loss': valid_loss,
        'accuracy': train_accuracy,
        'val_accuracy': valid_accuracy,
    })
    liveloss.draw()  
  print('Accuracy: {:.4f} (valid), {:.4f} (train)'.format(valid_accuracy, train_accuracy))
  return model # 学習したモデルを返す

　それでは、CNN＋LinearによるMNIST予測モデルを構築し、学習を行ってみよう。今回は `Conv2d()` を1回行うごとに `MaxPool2d()` で画像サイズを2分の1にしているが、複数回 `Conv2d()` をしてから`MaxPool2d`をすることもある（時間があれば、基盤人工知能 第7回 講義で紹介したAlexNetのネットワーク構造を見てみると良い）。

In [0]:
torch.manual_seed(0) # 学習結果の再現性を担保
torch.backends.cudnn.deterministic = True

cnn = torch.nn.Sequential(
    torch.nn.Conv2d(1, 16, (5, 5)),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Conv2d(16, 32, (5, 5)),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),    
    torch.nn.Flatten(), 
    torch.nn.Linear(32*4*4, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10),
)
cnn.to(device_GPU)

　上記のCNNの定義のコードの中には**入力画像サイズ（縦横28ピクセルずつ）が明示的に記述されていない**ことに注意してほしい。上記のモデルの各段階で画像データがどのような変遷をたどるか、**図AI7.8**に図解しておくので、こちらも併せてみておくと良いだろう。


![図 AI7.8](https://i.imgur.com/GfxYmY0.png)

**図 AI7.8 | 今回構築したネットワーク** 画像状の構造をしているデータは常に正方であるため、画像幅とチャネル数のみ値を記している。また、簡略化の為に活性化関数は直前の層と併せて記述した。


In [0]:
torch.manual_seed(0) # 学習結果の再現性を担保

# 誤差関数と最適化手法を準備
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn.parameters(), lr=0.1)

# 学習の実行
trained_model = train(cnn, loss_fn, optimizer, train_loader, valid_loader, device_GPU)

# 予測の実行
test_loss, test_accuracy = evaluate_model(cnn, loss_fn, test_loader, device_GPU)
print(test_loss)
print(test_accuracy)

　先ほどは、至極当然のようにmodelを作成したが、各層の入力サイズ・出力サイズを意識しながらモデルを構築することは案外難しい。`Flatten`の直前では**32チャネル×縦4マス×横4マス**になっている、ということは、一瞬見ただけではなかなか理解ができない。

　このような場合、torchsummaryというライブラリを用いることで、簡単に各層の出力サイズを見ることができる。実際に利用してみよう。

In [0]:
# Flattenまでを抜き出した途中までのモデル
cnn_part = torch.nn.Sequential(
    torch.nn.Conv2d(1, 16, (5, 5)),
    torch.nn.MaxPool2d(2),
    torch.nn.ReLU(),
    torch.nn.Conv2d(16, 32, (5, 5)),
    torch.nn.MaxPool2d(2),    
    torch.nn.ReLU(),
    torch.nn.Flatten()
)
cnn_part.to(device_GPU)

In [0]:
import torchsummary
torchsummary.summary(cnn_part, (1, 28, 28)) # ミニバッチを除いた入力の形状（チャネル数、縦横サイズ）を入力する

　このように、入力が1チャネル×縦28マス×横28マスだった場合の各層の出力サイズが表示され、Flattenの出力が512次元であることがわかる（図AI7.8と見比べてみよ）。Flattenの次のLinear層の入力次元数は512とすればよいようだ。

----
##### 課題 AI7.2

　間違ったモデルを組んだ状態で学習をさせようとするとどうなるだろうか。
予測まで行う説明で使ったモデルの最初の`Conv2d()`を`torch.nn.Conv2d(1, 16, (5, 5), padding=2)`とすることで（誤った）モデルを構成し、`train()`を実行せよ。この時、どのようなエラーが発生するだろうか。エラーメッセージを報告し、メッセージの内容を簡単に説明せよ（ヒント：`torch.addmm`とはどういう関数か？）。


```
cnn = torch.nn.Sequential(
    torch.nn.Conv2d(1, 16, (5, 5)), # ここを書き換えることで誤ったモデルを作成する
    torch.nn.MaxPool2d(2),
    torch.nn.ReLU(),
    torch.nn.Conv2d(16, 32, (5, 5)),
    torch.nn.MaxPool2d(2),    
    torch.nn.ReLU(),
    torch.nn.Flatten(), 
    torch.nn.Linear(32*4*4, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10),
)
```

------
##### 課題 AI7.3

　課題 AI7.2におけるエラーメッセージを参考に`torch.nn.Flatten()`直後の`torch.nn.Linear()`の入力次元数を修正することで、`train()`を行ってもエラーが発生せずに学習を行えるモデルを作成せよ。

演習の解答としては、`torch.nn.Linear(____, 256)` の下線部を埋めよ。


-----
##### 課題 AI7.4 **（授業後に修正あり）**

　課題 AI7.3で実装したモデルについて学習を実施せよ。レポートには以下のことを記せ。

1. 50エポック終了時のモデルの訓練データ、検証データ、テストデータに対する正解率
2. 説明に用いたモデルと課題 AI7.3のモデルは、どちらの方が性能が良かったか。

なお、この課題を行う際には、演習資料のコードと同様に、モデルの定義と学習の実行の前にそれぞれ `torch.manual_seed(0)` を~行い、 複数回実行しても実験結果が一致していることを確認せよ~行え。

（1/23補足情報：もしGPUを使ってでも結果を固定したい場合は、学習前に `torch.backends.cudnn.deterministic = True` とコードを記述すると良い。）

----
##### 課題 AI7.5（発展）

　研究において、「他人が再現実験が行えること」は非常に重要な要素である。
CNNモデルを自由に組み立て、学習を行った上で、テストデータの予測正解率を報告せよ。ただし、**採点者が全く同じ結果を得るために必要な情報を付与**せよ。

---

## AI7.2 | 再帰型ニューラルネットワーク (Recurrent Neural Network)

　**※ この節は授業では取り扱いません。各自実行してみてください。**

　RNN（Recurrent Neural Network、再帰型ニューラルネットワーク）は時系列データに対する逐次的な予測や、可変長のデータに対する単一の予測を行う際に利用できるニューラルネットワークの1手法である。

　近年ではRNNを発展させたLSTM-RNN (Long-Short term memory recurrent neural network)などが広く使われてはいるが、ここでは、最も基礎的なRNNを、実際に可変長のデータに対する予測問題を解くために利用してみよう。

### AI7.2.1 | データの準備：苗字から言語圏を予測する

　苗字のローマ字表記から、どの言語圏の苗字なのかを予測することを考える。
当然苗字は可変長なので、これについてもRNNなどの手法を用いることが妥当であろう。


　とりあえず、データを準備する（これはPyTorch公式のtutorialから取得している）。

In [0]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip -o data.zip

　いつものようにデータの中身を見てみよう。

In [0]:
!head -n10 data/names/Japanese.txt

#### RNNが理解できる表現への変換

　これまでと同様、深層学習自体は数値的な表現がされたデータしか学習や予測に用いることはできない。文字列は扱いに困るので、これをまず数値列に変換しよう。

　以下は苗字の文字列を数値列に変換するコードである（が、ここでは詳しくは説明しない。öをoに変換するなど、特殊な記号を排除した上で、苗字の文字列を`'A'->3, 'b'->30`といったように、文字と数値が1対1対応するように変換している）。

In [0]:
import glob
import numpy as np
import os
import string
import unicodedata

# Alphabet [a-zA-Z .,;']
alphabet = set(string.ascii_letters + " .,;'")

def normalize(s):
    # Apply canonical decomposition, and ignore non-alphabet symbols.
    return ''.join(
        c for c in unicodedata.normalize('NFD', s) if c in alphabet
        )
# normalize('Ślusàrski') -> 'Slusarski'

In [0]:
def build_char_mapping(surnames):
  characters = set()
  for surname in surnames:
    surname = normalize(surname)
    characters.update(list(surname))
  characters = sorted(characters)
  
  char_map = {}
  for i, char in enumerate(characters):
    char_map.setdefault(char, i)
  return char_map

def build_lang_mapping(languages):
  langs = set(languages)
  langs = sorted(langs)
  
  lang_map = {}
  for i, lang in enumerate(langs):
    lang_map.setdefault(lang, i)
  return lang_map


def convert_surnames(surnames, charmap):
  X = []
  for surname in surnames:
    chars_surname = list(surname)
    X.append([charmap[c] for c in chars_surname])
  return X

def convert_languages(languages, langmap):
  y = [langmap[s] for s in languages]
  return y

　さらに、18種類の言語圏を18種類のクラスラベル（数値）に変換する。

In [0]:
surnames = []
languages = []
srcs = glob.glob('data/names/*.txt')
srcs = sorted(srcs)
for src in srcs:
  lang = os.path.basename(src)[:-4]
  for line in open(src):
    line = line.strip('\n')
    surnames.append(normalize(line))
    languages.append(lang)

charmap = build_char_mapping(surnames)
langmap = build_lang_mapping(languages)

np.savez_compressed(
    'surname-language',
    charmap = charmap,
    langmap = langmap,
    surname = convert_surnames(surnames, charmap),
    language = convert_languages(languages, langmap)
)

　以上のコードで、苗字と言語圏をそれぞれ数値化したデータが完成した。
データの中身を見てみよう。

In [0]:
data = np.load('surname-language.npz', allow_pickle=True)

In [0]:
data["surname"]

In [0]:
X = data["surname"]
y = data["language"]

In [0]:
i = 9500
print(X[i])

　どうやら6文字の苗字のようだが・・・、人間には理解できなくなってしまった。復元してみよう。

In [0]:
# 復元用のdictデータの作成
rev_charmap = {} 
for key, value in charmap.items():
  rev_charmap[value] = key

In [0]:
i = 9500
print([rev_charmap[idx] for idx in data["surname"][i]])

　`Ochiai`さんが`[17, 31, 36, 37, 29, 37]`という名前に変換されていたようだ。

　そうしたら、いつものようにtorch.tensor化して、dataset, dataloaderを作成…したいのだが、**可変長な入力のままではDataLoaderを使うことができない。**今回は、DataLoaderを利用することをあきらめて、学習時に適宜処理することにする。

### AI7.2.2 | RNNを含んだモデルの構築と学習の実施

　次に、RNNを含んだモデルを構築するのだが、RNNを含んだモデル構築は多少技術が必要である。
というのも、一般的には**RNNは入力が与えられるたびに出力を行うモデル**である（**図 AI7.9**）ため、今回のように**文字列に対して1つの出力を得たい場合**には、**RNNの最後の出力のみを取り出して利用する**必要がある。

　このような仕組みは、これまで利用してきた`torch.nn.Sequential()`だけでは表現できず、自作する必要がある。


![図 AI7.9](https://i.imgur.com/M88oTpB.png)

**図 AI7.9 | RNNの模式図** 逐次的にデータが入力され、そのたびに出力が行われる。出力 $h$ は入力 $x$ だけではなく、それまでの入力列にも影響される。 



In [0]:
import torch

# 最後の出力だけを次の層に渡すRNN
class lastoutputRNN(torch.nn.Module):
  def __init__(self, input_size, output_size):
    super(lastoutputRNN, self).__init__() # 初期化のおまじない
    
    # 学習すべき重みがあるものをここに全て定義
    self.rnn = torch.nn.RNN(input_size, output_size, num_layers=1)
  
  # 予測時の計算を定義
  def forward(self, input):
    outs_rnn, hiddens = self.rnn(input, self.initHidden()) # ひとまず複数の 出力 h を全て受け取る
    last_out_rnn = outs_rnn[-1] # 最後の出力 h_r を抜き出す
    return last_out_rnn # 次の層に渡す

  # RNNの隠れ層の初期値を準備する補助関数
  def initHidden(self):
    return torch.zeros(1, 1, self.rnn.hidden_size)

　以上で用意した、RNNの最後の出力だけを出力する層を用いて、学習を行ってみよう（6分半程度かかるので、以下を実行してからコードを理解すると良い）。他人のコードを読むことは慣れている人間でもそれなりに時間がかかるので、落ち着いて理解していこう。

In [0]:
import json
import random
from livelossplot import PlotLosses # livelossplotのインストールを忘れずに（AI7.1.4参照）
import numpy as np

torch.manual_seed(0) # 実行結果を再現させる
np.random.seed(0)    # data_shuffle()のためにNumPyの乱数も指定する


###### Sequentialによるモデルの作成
# 入力：文字列のone-hot表現
# 出力：入力の文字列が「どの言語らしいか」を示す値（各言語ごとに実数値を出力）
model = torch.nn.Sequential(
    lastoutputRNN(len(charmap), 128), 
    torch.nn.Linear(128, len(langmap))
)
    
###### 入力・出力のPyTorch用データへの変換を行う関数
def x_to_tensor(x, input_size): # one hot encodingを行っている
    tensor = torch.zeros(len(x), 1, input_size, dtype=torch.float)
    for i, j in enumerate(x):
        tensor[i][0][j] = 1   # tensor.shape = (T, batch, input_dim)
    return tensor
def y_to_tensor(y):
    return torch.tensor([y], dtype=torch.long)
  
  
###### データをランダムに並び替える関数
# Xとyの対応関係を損なわないために、インデックスをshuffleして
# それに基づいてX, yを同時に並び変える、ということを行っている
def data_shuffle(X, y):
  indices = list(range(len(X)))
  np.random.shuffle(indices)
  X = X[indices]
  y = y[indices]
  return X, y

###### 入力 X と教師信号（期待される出力）y の読み込み
data = np.load('surname-language.npz', allow_pickle=True)
charmap = data["charmap"].item()
langmap = data["langmap"].item()
X = data["surname"]
y = data["language"]

###### 誤差関数と最適化手法の定義
# これはいつもと同じ
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)



###### 学習本体
liveloss = PlotLosses() # 描画の準備

for epoch in range(10): # 10 epoch 学習する
    total_train_loss = 0.0
    num_train_correct = 0
    X, y = data_shuffle(X, y)
    
    # Training loop for every instance.
    for Xi, yi in zip(X, y):
      # X, y からそれぞれ1つずつデータを取得する

        # 数値列データを逐次torch.tensorに変換
        Xi_torch = x_to_tensor(Xi, len(charmap))
        yi_torch = y_to_tensor(yi)

        # 予測の実施
        y_pred = model(Xi_torch) 
        _, predicted = torch.max(y_pred.data, 1)
        num_train_correct += (predicted == yi_torch).sum().item()
        
        # 予測の誤差を計算（交差エントロピー誤差）
        loss = loss_fn(y_pred, yi_torch)
        total_train_loss += loss.item()
        
        # 誤差逆伝播法による重みの更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # このepoch全体での予測精度 (accuracy) を計算
    train_accuracy = num_train_correct / len(y)

    # livelossplotの描画
    liveloss.update({
        'log loss': total_train_loss,
        'accuracy': train_accuracy,
    })
    liveloss.draw()
    
# 最後のepochにおける訓練データの予測精度を出力
print('Accuracy: {:.4f} (train)'.format(train_accuracy))

　このモデルを使うことで、15以上の言語圏の苗字を訓練データに対する予測精度で75%程度の正答率で予測することができた。学習曲線を見る限り、まだ精度は向上しそうである。

　最後に、このモデルを使って、`Masu`さんと`Sato`さんはどの言語圏の苗字かを予測してみよう。

In [0]:
new_surnames = ["Masu", "Sato"]
new_surnames = convert_surnames(new_surnames, charmap)  # 数値列化
for surname in new_surnames:
  x_torch = x_to_tensor(surname, len(charmap))
  pred = model(x_torch)                                 # 予測の実施
  _, predicted = torch.max(pred.data, 1)                # 最も確率の高かったクラスを求める
  print(predicted)                                      # 予測されたクラスを表示

　いずれもクラス10であると予測された。langmapを見ればクラス10はどの言語かがわかるので確認してみよう。

In [0]:
print(langmap)

　どうやら、正しく日本語の苗字だと認識したようだ（お分かりかと思うが、`Masu`は現在の東工大の学長の苗字である）。

------
##### 課題 AI7.6（発展、提出の必要はありません）

　このモデルをデータセットにない苗字に適用することを考える。その時の予測精度はどの程度であると予想されるか。以下の3択から選択し、その理由を簡単に述べよ。

1. 75%より高い精度で予測できると考えられる
2. 75%程度の精度で予測できると考えられる
3. 75%未満の精度で予測できると考えられる

-----
##### 課題 AI7.7（極めて発展、提出の必要はありません）
　`lastoutputRNN`クラスやモデルを適切に書き換えることで、苗字の途中まで読んだ時点での言語予想を出力させることができる。しかし、途中経過の予測結果が学習に影響を及ぼすのは好ましいことではない。そこで、**学習には影響させず、予測時のみ途中での言語予想を出力させる**ようなクラスおよびモデルを完成させよ。`Gotoh`（後藤）という苗字の言語予想がどのように変化していくか確認せよ。

-----

# レポート提出について



## レポートの提出方法
　レポートは**答案テンプレートを用い**、**1つのファイル (.doc, .docx, .pdf)** にまとめ、**学籍番号と氏名を確認の上**、**1/30 15:00 (次回 基盤人工知能演習) までに東工大ポータルのOCW-iから提出**すること。
ファイルのアップロード後、OCW-iで「提出済」というアイコンが表示されていることを必ず確認すること。それ以外の場合は未提出扱いとなるので十分注意すること。
また、締め切りを過ぎるとファイルの提出ができないため、時間に余裕を持って提出を行うこと。

### 答案テンプレート

```
学籍番号:
名前:

課題 AI7.1
 ____ チャネル × 縦 ____ ピクセル × 横 ____ ピクセル

課題 AI7.2
エラーメッセージ：
エラーの内容説明：

課題 AI7.3
torch.nn.Linear(____, 256)

課題 AI7.4
訓練データに対する正解率：
検証データに対する正解率：
テストデータに対する正解率：
より良いモデルはどちらか：

課題 AI7.5
（自由記述）

```